In [19]:
### code:

In [3]:

import copy
from queue import Queue, PriorityQueue
import sys
from tkinter import *
class Problem(object):

    def __init__(self, initial, goal=None):
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        '''
        Return a list of possible actions
        '''
        return [x for x in range(1,len(state)+1)]

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        new_state=copy.deepcopy(state)
        for r in range(len(new_state)):
            for c in range(len(new_state[0])):
                if new_state[r][c] == 0:
                    new_state[r][c]= action
                    return new_state

    def goal_test(self, state):
        '''
        Check is the current state is the goal state. Goal state is when the 
        board is completely filled and there are no conflicts
        '''

        if not self.is_filled(state):
            return False

        if self.check_rcs(state):
            return False

        if self.check_boxes(state):
            return False
        return True

    def prune(self,state):
        '''
        This function will return True is the current state of the board 
        can never be part of a solution. Faslse otherwise.
        '''
        return (self.check_boxes(state) or self.check_rcs(state))

    def is_filled(self,state):
        '''
        This function returns true is the board has no blank cells. False otherwise.
        '''
        for row in state:
            if 0 in row:
                return False
        return True

    def check_rcs(self,state):
        ''' 
        This funciton checks all rows and columns to see if there are any conflicts.
        If there is a conflict, the function will return True. False otherwise.
        '''

        # Checks for rows
        for r in range(len(state)):
            histroy_list = [False]*10
            for c in range(len(state[0])):
                if histroy_list[state[r][c]]:
                    return True
                else:
                    if state[r][c]!=0:
                        histroy_list[state[r][c]] = True


        # Checks for cols
        for c in range(len(state[0])):
            histroy_list = [False]*10
            for r in range(len(state)):
                if histroy_list[state[r][c]]:
                    return True
                else:
                    if state[r][c]!=0:
                        histroy_list[state[r][c]] = True

        return False

    def check_boxes(self,state):

        '''
        This function checks sub boxes in the board for any conflicts. Returns True 
        if there is a conflict and false otherwise.
        '''
        board_len = len(state)
        box_len   = len(state)//3

        for colstart in range(0,board_len,3):
            for box in range(0,board_len-1,box_len):
                histroy_list = [False]*10
                for r in range (box,box+box_len):
                    for c in range(colstart,colstart+3):
                        if histroy_list[state[r][c]]:
                            return True
                        else:
                            if state[r][c]!=0:
                                histroy_list[state[r][c]] = True

        return False

In [4]:


class Node:
    def __init__(self, state, parent=None, action=None):
        self.state = state
        self.parent = parent
        self.action = action
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def expand(self, problem):
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        next_state = problem.result(self.state, action)
        return Node(next_state, self, action)

    def __lt__(self, other):
        return self.depth < other.depth

    def __eq__(self, other):
        return self.depth == other.depth

    def __hash__(self):
        return hash(self.state)

    
 

In [5]:
   
    
def btr(problem):
    """
    Solves a Sudoku problem using backtracking and recursion,
    returning the solution node or None if no solution exists.

    Args:
        problem: A Problem object representing the Sudoku puzzle.

    Returns:
        A Node object representing the solution or None if no solution exists.
    """

    def is_safe(board, row, col, num):
        # Check row and column for duplicates
        for i in range(len(board)):
            if board[row][i] == num or board[i][col] == num:
                return False

        # Check 3x3 subgrid for duplicates
        box_size = int(len(board) ** 0.5)
        box_row = row // box_size
        box_col = col // box_size
        for i in range(box_row * box_size, box_row * box_size + box_size):
            for j in range(box_col * box_size, box_col * box_size + box_size):
                if board[i][j] == num:
                    return False

        return True

    def solve_sudoku(board, row, col, parent):
        if row == len(board):
            return Node(board, parent)

        if col == len(board[0]):
            return solve_sudoku(board, row + 1, 0, parent)

        if board[row][col] != 0:
            return solve_sudoku(board, row, col + 1, parent)

        for num in range(1, len(board) + 1):
            if is_safe(board, row, col, num):
                new_board = copy.deepcopy(board)
                new_board[row][col] = num
                child = solve_sudoku(new_board, row, col + 1, Node(board, parent))
                if child is not None:
                    return child

        return None  # Backtrack

    initial_board = copy.deepcopy(problem.initial)
    solution = solve_sudoku(initial_board, 0, 0, None)
    if solution is not None:
        problem.initial = solution.state  # Update initial state with solution
        return solution
    else:
        return None



In [6]:


def breadth_first_search(problem):
    node = Node(problem.initial)
    if problem.goal_test(node.state):
        return node

    frontier=Queue()
    frontier.put(node)

    while frontier.qsize()!=0:
        node = frontier.get()
        problem.initial=node.state

        for child in node.expand(problem):
            if problem.goal_test(child.state):
                return child

            # Pruneing
            if child != None and problem.prune(child.state) is False:
                frontier.put(child)

    return None

In [7]:


def depth_first_search(problem):
    """
    Solves a Sudoku problem using depth-first search.

    Args:
        problem: A Problem object representing the Sudoku puzzle.

    Returns:
        A Node object representing the solution or None if no solution exists.
    """
    node = Node(problem.initial)
    if problem.goal_test(node.state):
        return node

    frontier = [node]

    while frontier:
        node = frontier.pop()

        if problem.goal_test(node.state):
            return node

        for child in node.expand(problem)[::-1]:
            if not problem.prune(child.state):
                frontier.append(child)

    return None

greedy function:
Minimum Remaining Values (MRV): This heuristic selects the variable (cell) with the fewest remaining legal values. It prioritizes the variables that are likely to lead to a solution faster.

Cost function for UCS:
the number of constraints violated as the cost function

In [8]:
def uniform_cost_search(problem):
    """
    Solves a Sudoku problem using Uniform Cost Search (UCS).

    Args:
        problem: A Problem object representing the Sudoku puzzle.

    Returns:
        A Node object representing the solution or None if no solution exists.
    """
    node = Node(problem.initial)
    if problem.goal_test(node.state):
        return node

    frontier = PriorityQueue()  # Priority queue based on cost
    frontier.put((0, node))  # Initial cost is 0

    while not frontier.empty():
        cost, node = frontier.get()

        if problem.goal_test(node.state):
            return node

        for child in node.expand(problem):
            child_cost = calculate_cost(problem, child.state)
            frontier.put((child_cost, child))

    return None

def calculate_cost(problem, state):
    """
    Calculates the cost for a given Sudoku state based on the number of constraints violated.

    Args:
        problem: A Problem object representing the Sudoku puzzle.
        state: A 2D list representing the current state of the Sudoku puzzle.

    Returns:
        The cost of the state based on the number of constraints violated.
    """
    # Count the number of constraints violated
    constraints_violated = 0
    for r in range(len(state)):
        for c in range(len(state[0])):
            num = state[r][c]
            if num != 0:
                if not is_safe(state, r, c, num):
                    constraints_violated += 1

    return constraints_violated

def is_safe(state, row, col, num):
    """
    Checks if placing a number in a given cell is safe according to Sudoku rules.

    Args:
        state: A 2D list representing the current state of the Sudoku puzzle.
        row: The row index of the cell.
        col: The column index of the cell.
        num: The number to be placed in the cell.

    Returns:
        True if placing the number is safe, False otherwise.
    """
    # Check row and column for duplicates
    for i in range(len(state)):
        if state[row][i] == num or state[i][col] == num:
            return False

    # Check 3x3 subgrid for duplicates
    box_size = int(len(state) ** 0.5)
    box_row = row // box_size
    box_col = col // box_size
    for i in range(box_row * box_size, box_row * box_size + box_size):
        for j in range(box_col * box_size, box_col * box_size + box_size):
            if state[i][j] == num:
                return False

    return True


In [9]:
def solve_sudoku_greedy(problem):
    """
    Solves a Sudoku puzzle using a greedy search algorithm with the Minimum Remaining Values (MRV) heuristic.

    Args:
        problem: A Problem object representing the Sudoku puzzle.

    Returns:
        The solution grid if a solution exists, or None if no solution is found.
    """
    grid = problem.initial
    empty_cells = [(row, col) for row in range(len(grid)) for col in range(len(grid[0])) if grid[row][col] == 0]

    while empty_cells:
        # Select the empty cell with the fewest remaining legal values (MRV heuristic)
        selected_cell = min(empty_cells, key=lambda cell: count_legal_values(grid, cell))

        # Choose a value for the selected cell that does not violate Sudoku rules
        legal_values = get_legal_values(grid, selected_cell)
        if not legal_values:
            return None  # Backtrack if no legal values are found

        # Try each legal value for the selected cell
        for value in legal_values:
            new_state = problem.result(grid, (selected_cell[0], selected_cell[1], value))
            new_problem = Problem(new_state)
            solution = solve_sudoku_greedy(new_problem)
            if solution:
                return solution  # Solution found

        # Remove the selected cell from the list of empty cells
        empty_cells.remove(selected_cell)

    return grid if problem.goal_test(grid) else None



def count_legal_values(grid, cell):
    """Counts the number of legal values for an empty cell."""
    return len(get_legal_values(grid, cell))

def get_legal_values(grid, cell):
    """Returns the legal values for an empty cell."""
    row, col = cell
    row_values = set(grid[row])
    col_values = set(grid[i][col] for i in range(len(grid)))  # Update range to len(grid)
    box_values = set(grid[i][j] for i in range(row//3*3, row//3*3 + 3)
                                    for j in range(col//3*3, col//3*3 + 3))
    return set(range(1, 10)) - row_values - col_values - box_values



In [10]:
from queue import PriorityQueue

def solve_sudoku_a_star(problem):
    """
    Solves a Sudoku puzzle using A* search combining greedy search and uniform cost search.

    Args:
        problem: A Problem object representing the Sudoku puzzle.

    Returns:
        The solution grid if a solution exists, or None if no solution is found.
    """
    grid = problem.initial
    empty_cells = [(row, col) for row in range(len(grid)) for col in range(len(grid[0])) if grid[row][col] == 0]

    while empty_cells:
        # Select the empty cell with the fewest remaining legal values (MRV heuristic)
        selected_cell = min(empty_cells, key=lambda cell: count_legal_values(grid, cell))

        # Choose a value for the selected cell that minimizes the number of constraints violated
        legal_values = get_legal_values(grid, selected_cell)
        if not legal_values:
            return None  # Backtrack if no legal values are found

        # Try each legal value for the selected cell
        min_cost = float('inf')
        best_value = None
        for value in legal_values:
            new_state = problem.result(grid, (selected_cell[0], selected_cell[1], value))
            new_problem = Problem(new_state)
            child_cost = calculate_cost(problem, new_state)
            if child_cost < min_cost:
                min_cost = child_cost
                best_value = value

        # Update the Sudoku grid with the best value for the selected cell
        grid[selected_cell[0]][selected_cell[1]] = best_value

        # Remove the selected cell from the list of empty cells
        empty_cells.remove(selected_cell)

    return grid if problem.goal_test(grid) else None


In [11]:
from tkinter import messagebox

class  GUI:
    def __init__(self, n):
        self.board_size = n
        self.master = Tk()
        self.master.title("SUDOKU")
        self.create_board()

    def create_board(self):
        '''
        Funtion creates all text cells and buttons required
        '''
        self. cells = [[None for r in range(self.board_size)] for c in range(self.board_size)]
        for row in range(self.board_size):
            for col in range(self.board_size):
                self.cells[row][col] = Entry(self.master, width=3)
                self.cells[row][col].grid(row=row,column=col)
                
                
        self.solve_button = Button(self.master, text="Solve", width=10, command=self.solve, bg="orange", fg="dark green")
        self.clear_button = Button(self.master, text="Clear", width=10, command=self.clear, bg="orange", fg="dark green")
        self.slider = Scale(self.master, from_=6, to=15, orient=HORIZONTAL, bg="purple", fg="white")
        self.resize_button = Button(self.master, text="Resize", width=10, command=lambda: self.resize(self.slider.get()), bg="orange", fg="dark green")

        # Configure background color for the board
        self.master.configure(bg="dark blue")

        

        self.solve_button.grid(row=0,column=self.board_size+1)
        self.clear_button.grid(row=1,column=self.board_size+1)
        self.slider.grid(row=2, column=self.board_size+1)
        self.resize_button.grid(row=3,column=self.board_size+1)
        
        
        
        

        self.controls = [self.solve_button,self.clear_button,self.resize_button,self.slider]

    def get_values(self):
        '''
        Returns values from the board as a 2 dimensional array. 0 represents empty cells
        '''
        values = [[0 for r in range(self.board_size)] for c in range(self.board_size)]
        for row in range(self.board_size):
            for col in range(self.board_size):
                if self.cells[row][col].get():
                    values[row][col] = int(self.cells[row][col].get())
        return values

    def set_values(self, new_vals):
        '''
        Given a set of new values, function will write them on the GUI board. 
        Old values will be replaced.

        :param new_vals: (two-dimensional list) containing new values
        '''
        values = [[0 for r in range(self.board_size)] for c in range(self.board_size)]
        for row in range(self.board_size):
            for col in range(self.board_size):
                self.cells[row][col].delete(0, END)
                self.cells[row][col].insert(0,new_vals[row][col])

    def clear(self):
        '''
        Clear values from all cells
        '''
        values = [[0 for r in range(self.board_size)] for c in range(self.board_size)]
        for row in range(self.board_size):
            for col in range(self.board_size):
                self.cells[row][col].delete(0, END)

    def resize(self,val):
        '''
        Resize board to the value passed in.

        :param val: (int) size of board 
        '''
        self.destroy_all()
        self.board_size=val
        self.create_board()

    def destroy_all(self):
        '''
        Destroy all buttons,cells, and slider from the board
        '''
        for row in range(self.board_size):
            for col in range(self.board_size):
                self.cells[row][col].destroy()

        for control in self.controls:
            control.destroy()


    def solve(self):
        '''
        Driver function
        '''
        values = self.get_values()
        p = Problem(values)
        solution = btr(p)
        #solution = breadth_first_search(p)
        #solution=depth_first_search(p)
        #solution=uniform_cost_search(p)
        #solution=solve_sudoku_greedy(p)
        #solution=solve_sudoku_a_star(p)
        if solution:
            self.set_values(solution.state)
        else:
            messagebox.showinfo("NA")

        


In [ ]:
# using BFS()
def main():
    board = GUI(6)
    mainloop()

if __name__ == "__main__":
    main()

# 